# The Finnis-Sinclair potential

Structure:
1. Computing Finnis-Sinclair forces
2. Computing Finnis-Sinclair potential energies

TODOs:
* test forces for all potentials using perfect crystals and crystals with a vacancy
* add tests for potential energy values for each structure
* other force implementation that does not require altering `function accelerations`?

In [ ]:
import Pkg

In [ ]:
Pkg.activate(".")

In [ ]:
Pkg.status()

In [ ]:
using DataFrames
using Molly
using Plots
using Test
using LaTeXStrings
using LinearAlgebra
using SparseArrays
using Crystal

## Parameterisation of V, Nb, Ta, Cr, Mo, W, Fe

Parameterisation by Finnis et al. 1984, _A simple empirical N-body potential for transition metals_

| element | d | A | $\beta$ | c | $c_0$ | $c_1$ | $c_2$ | 
| --- | --- | --- | --- | --- | --- | --- | --- |
| V  | 3.692767 | 2.010637 | 0   | 3.8  | -0.8816318 | 1.4907756   | -0.3976370 |
| Nb | 3.915354 | 3.013789 | 0   | 4.2  | -1.5640104 | 2.0055779   | -0.4663764 |
| Ta | 4.076980 | 2.591061 | 0   | 4.2  | 1.2157373  | 0.0271471   | -0.1217350 |
| Cr | 3.915720 | 1.453418 | 1.8 | 2.9  | 29.1429813 | -23.3975027 | 4.7578297 |
| Mo | 4.114825 | 1.887117 | 0   | 3.25 | 43.4475218 | -31.9332978 | 6.0804249 |
| W  | 4.400224 | 1.896373 | 0   | 3.25 | 47.1346499 | -33.7665655 | 6.2541999 |
| Fe | 3.699579 | 1.889846 | 1.8 | 3.4  | 1.2110601  | -0.7510840  | 0.1380773 |

In [ ]:
elements = ["V", "Nb", "Ta", "Cr", "Mo", "W", "Fe"]
element_pairings = [string(el,el) for el in elements]
element_pair_map = Dict(pair => i for (i,pair) in enumerate(element_pairings))

In [ ]:
df = DataFrame(
    element_pair = element_pairings,
    d = [3.692767, 3.915354, 4.076980, 3.915720, 4.114825, 4.400224, 3.699579],
    A = [2.010637, 3.013789, 2.591061, 1.453418, 1.887117, 1.896373, 1.889846],
    β = [0, 0, 0, 1.8, 0, 0, 1.8],
    c = [3.8, 4.2, 4.2, 2.9, 3.25, 3.25, 3.4],
    c₀ = [-0.8816318, -1.5640104, 1.2157373, 29.1429813, 43.4475218, 47.1346499, 1.2110601],
    c₁ = [1.4907756, 2.0055779, 0.0271471, -23.3975027, -31.9332978, -33.7665655, -0.7510840],
    c₂ = [-0.3976370, -0.4663764, -0.1217350, 4.7578297, 6.0804249, 6.2541999, 0.1380773],
)

In [ ]:
masses = Dict("V" => 50.9415, "Nb" => 92.9064, "Ta" => 180.9479,
              "Cr" => 51.996, "Mo" => 95.94, "W" => 183.85,
              "Fe" => 55.847)

In [ ]:
# Å
bcc_lattice_constants = Dict(
    "V" => 3.0399, "Nb" => 3.3008, 
    "Ta" => 3.3058, "Cr" => 2.8845, "Mo" => 3.1472, 
    "W" => 3.1652, "Fe" => 2.8665
)

## Interaction

In [ ]:
"""
A glue interaction that will apply to all atom pairs.
Finnis-Sinclair and similar interactions should sub-type this type.
This type should be a GeneralInteraction type. But due to the special
nature of glue interactions and the restriction to pair interactions
of the GeneralInteraction type, glue interactions are for now a sub-type of 
SpecificInteraction.
"""
abstract type GlueInteraction <: SpecificInteraction end

In [ ]:
struct FinnisSinclairInteraction <: GlueInteraction
    nl_only::Bool
    element_pair_map::Dict
    params::DataFrame
end

Instantiating the interaction

In [ ]:
fs_inter = FinnisSinclairInteraction(true, element_pair_map, df)

## Glue potential

The glue potential is the core component which makes the Finnis-Sinclair empirical potential and other similar approaches different to, for example, the Lennard-Jones potential. 
TODO: some more explanation

$$
\phi(r) = (r-d)^2 + \beta (r-d)^3/d
$$

In [ ]:
function glue_potential(r::T, β::T, d::T)::T where T<:Real
    return r > d ? 0 : (r-d)^2 + β*(r-d)^3/d
end

In [ ]:
element_pair, d, A, β, c, c₀, c₁, c₂ = df[1,:] # parameters for Vanadium

In [ ]:
r = collect(range(0, stop=2*3.3058, length=1000));
ɸ = glue_potential.(r, β, d);

In [ ]:
ɸs = [ɸ]
element_pairs = [element_pair]
for i in 2:nrow(df)
    element_pair, d, A, β, c, c₀, c₁, c₂ = df[i,:]
    ɸ = glue_potential.(r, β, d)
    append!(ɸs,[ɸ])
    element_pairs = hcat(element_pairs, string(element_pair))
end

In [ ]:
plot(r, ɸs, label=element_pairs)

## Computing forces

This is what we need for the simulation.

$\partial_k = \partial_{R_k} = $ change of atom position $k$, $r_{ij} = \|R_{ij}\|_2$, $R_{ij} = R_j - R_i \in \mathbb{R}^3$ 

$$
\partial_k u = \partial_k u_\text{pair} + \partial_k u_\text{glue} 
$$

$$
\partial_k u_\text{pair} = \sum_{i>j} V_{ij}^\prime(r_{ij})\partial_k r_{ij}
$$

$$
\partial_k u_\text{glue} = \sum_i f_i^\prime(\rho_i) \cdot \partial_k \rho_i 
$$

$$
V_{ij}^\prime(r_{ij}) = 2(r-c)(c_0 + c_1 r + c_2 r^2) + (r-c)^2(c_1 + 2c_2r)
$$

$$
f_i^\prime(\rho_i) \cdot \partial_k \rho_i =
\begin{cases}
    k = i, & f_k^\prime(\rho_k) \sum_j\phi_j^\prime(r_{kj})\partial_k r_{kj} \\
    k \ne i, & \sum_{i\ne k} f_i^\prime(\rho_i) \partial_k \phi_k^\prime(r_{ik})\partial_k r_{ik} \\
\end{cases}
$$

$$
f_i^\prime = \frac{1}{2}A_i\rho_i^{-3/2}
$$

$$
\partial_k\phi(r) = \left[2(r-d) + 3\beta (r-d)^2/d\right] \cdot
\begin{cases}
    k = i, &\frac{R_{kj}}{r_{kj}} \\ 
    k = j, &\frac{R_{ik}}{r_{ik}} \\
\end{cases}
$$

$$
\partial_k r_{ij} =
\begin{cases}
    k = i, &\frac{R_{kj}}{r_{kj}} \\ 
    k = j, &\frac{R_{ik}}{r_{ik}} \\
\end{cases}
$$

In [ ]:
function pair_potential_derivative(r::T, c::T, c₀::T, c₁::T, c₂::T)::T where T<:Real
    return (r > c) ? 0 : 2 * (r - c) * (c₀ + c₁*r + c₂*r^2) + (r - c)^2 * (c₁ + 2*c₂*r)
end

function glue_energy_derivative(ρ::Float64, A::Float64)::Float64
   return A/2 * ρ^-1.5 
end

function glue_potential_derivative(r::T, β::T, d::T)::T where T<:Real
    return r > d ? 0 : 2*(r-d) + 3*β*(r-d)^2/d
end

In [ ]:
function get_pair_params(element1::String, element2::String, inter::FinnisSinclairInteraction)
    pair = string(sort([element1, element2])...)
    return inter.params[inter.element_pair_map[pair],:]
end

# @inline @inbounds 
function force(
        inter::FinnisSinclairInteraction, 
        coords, 
        s #::MinimalSimulationConfig
    )
    # computing the embedding densities
    n_atoms = length(s.coords)
    ρs = zeros(n_atoms)
    rs = zeros(length(s.neighbours))
    r_vec_norms = zeros(length(s.neighbours),3)
    
    for (n,(i,j)) in enumerate(s.neighbours)
        element_i = s.atoms[i].name
        element_j = s.atoms[j].name
        element_pair = string(sort([element_i, element_j])...)
        pi = get_pair_params(element_i,element_i,inter) # inter.params[inter.element_map[element_i],:]
        pj = get_pair_params(element_j,element_j,inter) # inter.params[inter.element_map[element_j],:]
        pij = get_pair_params(element_i,element_j,inter) # inter.params[inter.element_map[element_pair],:]
        
        r_vec = vector(s.coords[i], s.coords[j], s.box_size)
        r2 = sum(abs2, r_vec)
        r = sqrt(r2)
        # storing distance (vectors) so we don't need to recompute
        rs[n] = r
        r_vec_norms[[n],:] = r_vec / r
        # storing glue densities
        ρs[i] += glue_potential(r, pj.β, pj.d)
        ρs[j] += glue_potential(r, pi.β, pi.d)
    end
    
    fs = [zeros(1,3) for _ in 1:n_atoms]
    for (n,(i,j)) in enumerate(s.neighbours)
        element_i = s.atoms[i].name
        element_j = s.atoms[j].name
        element_pair = string(sort([element_i, element_j])...)
        pi = get_pair_params(element_i,element_i,inter) # inter.params[inter.element_map[element_i],:]
        pj = get_pair_params(element_j,element_j,inter) # inter.params[inter.element_map[element_j],:]
        pij = get_pair_params(element_i,element_j,inter) # inter.params[inter.element_map[element_pair],:]
        
        r = rs[n]
        r2 = r^2
        r_vec_norm = r_vec_norms[[n],:]
        
        # pair contribution
        dpairdR_i = r_vec_norm * pair_potential_derivative(r, pij.c, pij.c₀, pij.c₁, pij.c₂)
        dpairdR_j = - dpairdR_i
        
        # glue contribution
        dudρ_i = glue_energy_derivative(ρs[i], pi.A)
        dudρ_j = glue_energy_derivative(ρs[j], pj.A)
        dΦdr_i = glue_potential_derivative(r, pi.β, pi.d)
        dΦdr_j = glue_potential_derivative(r, pj.β, pj.d)
        
        ## density change by moving the current atom
        dgluedR_i_curr = r_vec_norm * dudρ_i * dΦdr_j
        dgluedR_j_curr = r_vec_norm * dudρ_j * dΦdr_i
        ## density change by moving a neighbouring atom
        dgluedR_i_neigh = - r_vec_norm * dudρ_j * dΦdr_i
        dgluedR_j_neigh = - r_vec_norm * dudρ_i * dΦdr_j
        
        # updating the forces
        f_i = (dpairdR_i + dgluedR_i_curr + dgluedR_i_neigh)
        f_j = (dpairdR_j + dgluedR_j_curr + dgluedR_j_neigh)
#         println("\nf_i ", f_i)
#         println("f_j ", f_j)
        fs[i] += f_i
        fs[j] += f_j
    end
    
    return collect(1:n_atoms), fs
end

In [ ]:
function Molly.force(
        inter::FinnisSinclairInteraction, 
        coords, 
        s #::MinimalSimulationConfig
    )
    return force(inter, coords, s)
end

In [ ]:
element = "Fe"
a = bcc_lattice_constants[element]
atoms, coords, box, box_size, box_vectors = Crystal.make_bcc_unitcell(element, a=a)
sc_atoms, sc_coords, sc_box, sc_box_size = Crystal.make_supercell(atoms, coords, box, box_size, nx=3, ny=3,nz=3)
n_atoms = length(sc_atoms);

In [ ]:
specific_inter_list = ((fs_inter,),)
velocities = [velocity(1., .01, dims=3) for i in 1:n_atoms]
sim = VelocityVerlet()
nb_matrix = trues(n_atoms,n_atoms)
n_steps = 1
dist_cutoff = 2 * a

nf = DistanceNeighbourFinder(nb_matrix, n_steps, dist_cutoff);

loggers = Dict("temperature" => TemperatureLogger(1))

In [ ]:
s = Simulation(
    simulator=sim, 
    atoms=sc_atoms, 
    specific_inter_lists=specific_inter_list,
    general_inters=(),
    coords=[SVector{3}(v) for v in sc_coords], 
    velocities=velocities,
    temperature=.01, 
    box_size=sc_box_size[1,1],
    timestep=.002,
    n_steps=10,
    neighbour_finder=nf,
    loggers=loggers,
)

In [ ]:
sparse_forces = force.((fs_inter,), (s.coords,), (s,))

Testing that all forces are about 0

In [ ]:
function test_sparse_forces_zero(sparse_forces, n_atoms; dims=3)
    zeros = [zero(rand(1,3)) for _ in 1:n_atoms]
    forces = getindex.(sparse_forces,2)[1]
    return all(isapprox.(forces, zeros, atol=1e-6))
end

In [ ]:
@test test_sparse_forces_zero(sparse_forces, n_atoms)

Modifying `Molly.accelerations` so the forces from the glue interaction are properly used to update the atom forces.

In [ ]:
function Molly.accelerations(s::Simulation; parallel::Bool=true)
    n_atoms = length(s.coords)

    if parallel && nthreads() > 1 && n_atoms >= 100
        forces_threads = [zero(s.coords) for i in 1:nthreads()]

        # Loop over interactions and calculate the acceleration due to each
        for inter in values(s.general_inters)
            if inter.nl_only
                neighbours = s.neighbours
                Threads.@threads for ni in 1:length(neighbours)
                    i, j = neighbours[ni]
                    force!(forces_threads[threadid()], inter, s, i, j)
                end
            else
                Threads.@threads for i in 1:n_atoms
                    for j in 1:(i - 1)
                        force!(forces_threads[threadid()], inter, s, i, j)
                    end
                end
            end
        end

        forces = sum(forces_threads)
    else
        forces = zero(s.coords)

        for inter in values(s.general_inters)
            if inter.nl_only
                neighbours = s.neighbours
                for ni in 1:length(neighbours)
                    i, j = neighbours[ni]
                    force!(forces, inter, s, i, j)
                end
            else
                for i in 1:n_atoms
                    for j in 1:(i - 1)
                        force!(forces, inter, s, i, j)
                    end
                end
            end
        end
    end

    for inter_list in values(s.specific_inter_lists)
        sparse_forces = force.(inter_list, (s.coords,), (s,))
        ge1, ge2 = getindex.(sparse_forces, 1), getindex.(sparse_forces, 2)
        sparse_vec = SparseVector(n_atoms, reduce(vcat, ge1), reduce(vcat, ge2))
        if typeof(inter_list[1]) == FinnisSinclairInteraction
            forces += Array([SVector{3}(v) for v in sparse_vec])
        else
            forces += Array(sparse_vec)
        end
        
    end

    for i in 1:n_atoms
        forces[i] /= s.atoms[i].mass
    end

    return forces
end

In [ ]:
function test_forces_zero(forces, n_atoms; dims=3)
    zeros = [zero(rand(3)) for _ in 1:n_atoms]
    return all(isapprox.(forces, zeros, atol=1e-6))
end

In [ ]:
@test test_forces_zero(accelerations(s, parallel=false), n_atoms)

In [ ]:
simulate!(s, parallel=false)

In [ ]:
s.loggers["temperature"].temperatures

## Computing energies

This is only really interesting for logging / development of potentials.

### Pair energy

$$
V_{ij}(r_{ij}) = 
\begin{cases} 
r \le c, & (r-c)^2 \left( c_0 + c_1 r + c_2 r^2 \right) \\
r > c, & 0 \\
\end{cases}
$$


In [ ]:
function pair_potential(r::T, c::T, c₀::T, c₁::T, c₂::T)::T where T<:Real
    return (r > c) ? 0 : (r - c)^2 * (c₀ + c₁*r + c₂*r^2)
end

In [ ]:
element_pair, d, A, β, c, c₀, c₁, c₂ = df[1,:] # parameters for Vanadium

In [ ]:
V = pair_potential.(r, c, c₀, c₁, c₂);

In [ ]:
Vs = [V]
element_pairs = [element_pair]
for i in 2:nrow(df)
    element_pair, d, A, β, c, c₀, c₁, c₂ = df[i,:]
    V = pair_potential.(r, c, c₀, c₁, c₂)
    append!(Vs,[V])
    element_pairs = hcat(element_pairs, string(element_pair))
end

In [ ]:
plot(r, Vs, label=element_pairs)

### Glue energy

Computing an energy based on local glue values

$$
u_\text{glue} = -A \cdot \sqrt{\rho}
$$

$$
\rho = \sum_{j \in \text{neighborhood}(i)} \phi(r_{ij})
$$

In [ ]:
function glue_energy(ρ::Float64, A::Float64)::Float64
   return -A * √ρ 
end

In [ ]:
ρ = 4. # density that you get summing phi-contributions from neighbours
glue_energy(ρ, 1.)

In [ ]:
ρ = collect(range(0, stop=50, length=100));

A = df.A[1] # Va
uₙ = glue_energy.(ρ, A)
element_pair = df.element_pair[1]

In [ ]:
uₙs = [uₙ]
element_pairs = [element_pair]
for i in 2:nrow(df)
    element_pair, d, A, β, c, c₀, c₁, c₂ = df[i,:]
    uₙ = glue_energy.(ρ, A)
    append!(uₙs,[uₙ])
    element_pairs = hcat(element_pairs, string(element_pair))
end

In [ ]:
plot(ρ, uₙs, label=element_pairs)

### Pair + glue energy = magic

$$ u_\text{tot} = u_N + u_P $$

$$ u_P = \frac{1}{2}\sum_{i=1,j=1}^{n_\text{atoms},n_\text{atoms}} V(r_{ij}) $$

$$ u_N = \sum_{i=1}^{n_\text{atoms}} u_\text{glue}(\rho_i) $$

In [ ]:
# @inline @inbound 
function potential_energy(inter::FinnisSinclairInteraction, s) # ::Simulation
    #computing the potential energy combining glue and pair components.
    
    #U = eltype(s.coords[i])
    #i == j && return zero(U) # ?
    e_pair = 0.
    e_glue = 0.
    n_atoms = length(s.coords)
    ρs = zeros(n_atoms)
    for (i,j) in s.neighbours
        element_i = s.atoms[i].name
        element_j = s.atoms[j].name
        element_pair = string(sort([element_i, element_j])...)
        pi = get_pair_params(element_i,element_i,inter) # inter.params[inter.element_map[element_i],:]
        pj = get_pair_params(element_j,element_j,inter) # inter.params[inter.element_map[element_j],:]
        pij = get_pair_params(element_i,element_j,inter) # inter.params[inter.element_map[element_pair],:]
        
        r_vec = vector(s.coords[i], s.coords[j], s.box_size)
        r2 = sum(abs2, r_vec)
        r = sqrt(r2)
        
        e_pair += pair_potential(r, pij.c, pij.c₀, pij.c₁, pij.c₂)
        
        ρs[i] += glue_potential(r, pj.β, pj.d)
        ρs[j] += glue_potential(r, pi.β, pi.d)
    end
    
    es_glue = zeros(n_atoms)
    for (i, atom) in enumerate(s.atoms)
        A = get_pair_params(atom.name, atom.name, inter).A
        es_glue[i] = glue_energy(ρs[i], A)
    end
    e_glue = sum(es_glue)
#     println("e_pair: ", e_pair, " e_glue: ", e_glue, "\n\nes_glue: ", es_glue, "\n\nρs: ", ρs)
    return e_pair + e_glue 
end

In [ ]:
function Molly.potential_energy(inter::FinnisSinclairInteraction, s)
    return potential_energy(inter, s)
end

In [ ]:
u = Molly.potential_energy(fs_inter, s) / n_atoms

In [ ]:
@test isapprox(u, -4.28, atol=1e-2)

### Running simulation with potential energy logger

In [ ]:
loggers = Dict(
    "temperature" => TemperatureLogger(1),
    "energy" => EnergyLogger(1),
)

In [ ]:
s = Simulation(
    simulator=sim, 
    atoms=sc_atoms, 
    specific_inter_lists=specific_inter_list,
    general_inters=(),
    coords=[SVector{3}(v) for v in sc_coords], 
    velocities=velocities,
    temperature=.01, 
    box_size=sc_box_size[1,1],
    timestep=.002,
    n_steps=10,
    neighbour_finder=nf,
    loggers=loggers,
)

In [ ]:
simulate!(s, parallel=false)

In [ ]:
s.loggers["energy"].energies